### Step1. Ready for Tranfer Learning

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [1]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52), #이미지 일부를 랜덤하게 잘라내어 52*52 사이즈로 변경
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], #입력 데이터 정규화(평균, 표준편차)
                             [0.229, 0.224, 0.225]) #데이터 정규화는 모델을 최적화하며 Local Minimum에 빠지는 것 방지
    ]),
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [ ]:
data_dir = './splitted'

#datasets 준비 + transform
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x])
                  for x in ['train', 'val']}
                  
#미니 배치로 분리하는 역할
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                  batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

### Step2. Load Pre-Trained Model

In [ ]:
from torchvision import models

In [ ]:
#1. Pre-Trained 모델 불러오기
resnet = models.resnet50(pretrained=True) 
#pretrained=True: 미리 학습된 모델의 파라미터 값 그대로 가져옴
#          =False: 모델의 구조만 가져오고 파라미터 값 랜덤으로 설정

num_ftrs = resnet.fc.in_features  
resnet.fc = nn.Linear(num_ftrs, 33) #우리 주제에 맞는 채널의 수를 출력하는 layer 추가 
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters(), lr=0.001))
'''
일부 layer의 파라미터만을 업데이트하기 위해
requires_grad = True 로 설정된 layer의 파라미터에만 적용
'''

In [ ]:
from torch.optim import lr_scheduler
#lr_scheduler.StepLR()
#Epoch 따라 lr을 변경하는 역할, 즉 gamma만큼 곱해 lr을 감소

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)